In [19]:
import os
import re

import music21

ct_melody_fp = input("Input file or directory of files to convert from cT melody format to musicXML: ")

if os.path.isdir(ct_melody_fp):
    file_path_list = [os.path.join(ct_melody_fp, path) for path in os.listdir(ct_melody_fp)]
else:
    file_path_list = [ct_melody_fp]

Input file or directory of files to convert from cT melody format to musicXML: C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody


In [20]:
save_dir = input("Input the directory to save the generated musicxml files: ")
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

Input the directory to save the generated musicxml files: C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody_transposed_musicxml


In [21]:
key_to_convert = input("Input key you would like to convert to (Hit enter to use the original key): ").strip()

Input key you would like to convert to (Hit enter to use the original key): C


In [22]:
# This accepts key signatures with lowercase values (c# and C# etc.)
key_signature_regex = re.compile(r"\[([A-G]|[a-g])[#b]?\]")

# Note this accepts mode signatures outside the specification of only 2, 3, 5, 6, or 7
# listed in the Temperley file format as being able to be flattened
mode_signature_regex = re.compile(r"\[[.b]{7}\]")

# Note that this matches any non-negative integer values including those with leading 0's
time_signature_regex = re.compile(r"\[\d+/\d+\]")

# Note this matches negative values and values with leading zeroes
octave_regex = re.compile(r"\[OCT=-?\d+\]")

unclear_meter_regex = re.compile(r"\[0\]")

# Exactly matches strings of the format "R*N" where N is a decimal value
multiple_measure_rest = re.compile(r"\AR\*\d+\Z")

# Exactly matches single R
single_measure_rest = re.compile(r"\AR\Z")

octave_up_regex = re.compile(r"\^")

octave_down_regex = re.compile(r"v")

sharpened_regex = re.compile(r"#")

flattened_regex = re.compile(r"b")

normalized_regex = re.compile(r"n")

note_onset_regex = re.compile(r"[1-7]")

no_onset_regex = re.compile(r"\.")


regex_list = [
    key_signature_regex,
    mode_signature_regex,
    time_signature_regex,
    octave_regex,
    unclear_meter_regex,
    multiple_measure_rest,
    single_measure_rest,
    octave_up_regex,
    octave_down_regex,
    sharpened_regex,
    flattened_regex,
    normalized_regex,
    note_onset_regex,
    no_onset_regex
]

note_count_regex_list = [
    mode_signature_regex,
    time_signature_regex,
    octave_regex,
    multiple_measure_rest
]

In [23]:
def convert_to_musicxml(melody_file, transpose_key=None):
    scale = music21.scale.MajorScale(music21.pitch.Pitch("C"))
    prev_note = None
    num_beats = 4
    base_note = 4
    octave = 4
    mode = [False for _ in range(7)]
    score = music21.stream.Score()
    melody = music21.stream.Part()
    # TODO, add metadata etc to score and part
    
    with open(melody_file) as f_in:
        song_text = f_in.read()
        
    lines = song_text.split("\n")
    
    # Cut out comments
    lines = [line.split("%")[0] for line in lines]
    
    # Make lines one long string
    data_stream = "".join(lines)
    
    # Cut out whitespace
    data_stream = "".join(data_stream.split())
    
    # Expand no onsets
    data_stream = data_stream.replace("-", "...")
    data_stream = data_stream.replace("_", "....")
    data_stream = re.sub(r"(?<!\[OCT)=+(?!\d+\])", "......", data_stream)
    
    # Remove ambiguity of dot notated rests vs. no onset
    data_stream = data_stream.replace("|.|", "|R|")

    # Transpose key if desired
    if transpose_key:
        scale = music21.scale.MajorScale(music21.pitch.Pitch(transpose_key))
        data_stream = re.sub(r"\[([A-G]|[a-g])[#b]?\]", "[{}]".format(transpose_key), data_stream)
    
    # Cut off the last item in the list because it is a "False" empty measure and split into measures
    measures = data_stream.split("|")[:-2]
    
    for measure_data in measures:
        new_measure = music21.stream.Measure()
        num_repeats = 1
        octave_modifier = 0
        accidental_alter = 0
        normalized = False
        
        if measure_data == "":
            # Handle the case where the measure is empty, meaning insert a rest measure
            time_sig = music21.meter.TimeSignature("/".join([str(num_beats), str(base_note)]))
            r = music21.note.Rest(quarterLength=4 * (num_beats / base_note))

            new_measure.append(time_sig)
            new_measure.append(r)
            
        else:
            # Count the number of items the measure is divided into to determine base note length
            num_notes = 0
            char_index = 0
            while char_index < len(measure_data):
                # First check all the tokens that use dots or numbers when they aren't specifying notes
                regex_match = None
                for regex in note_count_regex_list:
                    regex_match = regex.match(measure_data[char_index:])
                    if regex_match:
                        # By simply incrementing the index past these tokens we skip over false catches
                        char_index += regex_match.end()
                        break
                if regex_match is None:
                    # For the remaining tokens that must be notes add one to the note number
                    if measure_data[char_index] in {".", "1", "2", "3", "4", "5", "6", "7"}:
                        num_notes += 1
                        char_index += 1
                    # Any other non-note characters just skip over
                    else:
                        char_index += 1

            while measure_data != "":
                regex_match = None
                regex_index = -1
                for i, regex in enumerate(regex_list):
                    regex_match = regex.match(measure_data)
                    if regex_match:
                        regex_index = i
                        break

                if regex_index == -1:
                    raise ValueError("Error, unable to parse measure line")

                token = measure_data[regex_match.start():regex_match.end()]

                if regex_index == 0:
                    scale = music21.scale.MajorScale(music21.pitch.Pitch(token.strip("[]")))
                    mode = [False for _ in range(7)]

                elif regex_index == 1:
                    mode = [True if mode_val == "b" else False for mode_val in token.strip("[]")]

                elif regex_index == 2:
                    num_beats, base_note = map(int, token.strip("[]").split("/"))

                elif regex_index == 3:
                    octave = int(token.strip("[OCT=]"))
                    prev_note = None

                elif regex_index == 4:
                    print("Encountered unclear meter, defaulting to most recent time signature")

                elif regex_index == 5:
                    time_sig = music21.meter.TimeSignature("/".join([str(num_beats), str(base_note)]))
                    r = music21.note.Rest(quarterLength=4 * (num_beats / base_note))

                    new_measure.append(time_sig)
                    new_measure.append(r)
                    
                    num_repeats = int(token.strip("R*"))

                elif regex_index == 6:
                    time_sig = music21.meter.TimeSignature("/".join([str(num_beats), str(base_note)]))
                    r = music21.note.Rest(quarterLength=4 * (num_beats / base_note))

                    new_measure.append(time_sig)
                    new_measure.append(r)

                elif regex_index == 7:
                    octave_modifier += 1

                elif regex_index == 8:
                    octave_modifier -= 1

                elif regex_index == 9:
                    accidental_alter += 1

                elif regex_index == 10:
                    accidental_alter -= 1

                elif regex_index == 11:
                    normalized = True

                elif regex_index == 12:
                    if not new_measure.timeSignature:
                        time_sig = music21.meter.TimeSignature("/".join([str(num_beats), str(base_note)]))
                        new_measure.append(time_sig)
                        
                    scale_degree = int(token)

                    pitch_shift = music21.pitch.Accidental()
                    if mode[scale_degree-1]:
                        accidental_alter -= 1
                        
                    if normalized:
                        pitch_shift.alter = 0
                    else:
                        if accidental_alter != 0:
                            pitch_shift.alter = accidental_alter
                        else:
                            pitch_shift = None
                            
                    pitch = scale.pitchFromDegree(scale_degree)
                    
                    argument_dict = {
                        "name": pitch.name,
                        "quarterLength": (4 * num_beats / (base_note * num_notes)),
                        "octave": octave,
                    }
                    if pitch_shift is not None:
                        argument_dict["accidental"] = pitch_shift

                    if prev_note is not None:
                        midi_anchor = prev_note.pitch.midi
                        
                        same_octave_note = music21.note.Note(**argument_dict)
                        argument_dict["octave"] += 1
                        octave_up_note = music21.note.Note(**argument_dict)
                        argument_dict["octave"] -= 2
                        octave_down_note = music21.note.Note(**argument_dict)
                        
                        r = same_octave_note
                        same_octave_dist = abs(midi_anchor - same_octave_note.pitch.midi)
                        octave_up_dist = abs(midi_anchor - octave_up_note.pitch.midi)
                        octave_down_dist = abs(midi_anchor - octave_down_note.pitch.midi)
                        
                        shortest_dist = same_octave_dist

                        if octave_up_dist <= shortest_dist:
                            r = octave_up_note
                            shortest_dist = octave_up_dist
                            
                        if octave_down_dist < shortest_dist:
                            r = octave_down_note
                        
                    else:
                        r = music21.note.Note(**argument_dict)

                    r.octave += octave_modifier
                    new_measure.append(r)
                    
                    octave_modifier = 0
                    accidental_alter = 0
                    normalized = False
                    
                    prev_note = r
                    octave = r.octave
                        
                elif regex_index == 13:
                    if not new_measure.timeSignature:
                        time_sig = music21.meter.TimeSignature("/".join([str(num_beats), str(base_note)]))
                        new_measure.append(time_sig)

                    r = music21.note.Rest(quarterLength=(4 * num_beats / (base_note * num_notes)))

                    new_measure.append(r)
                
                measure_data = measure_data[regex_match.end():]
                
        melody.repeatAppend(new_measure, num_repeats)
        
    score.append(melody)
    
    gex = music21.musicxml.m21ToXml.GeneralObjectExporter(score)
    
    out = gex.parse()
    return out.decode('utf-8').strip()



In [24]:
for file in file_path_list:
    print(file)
    musicxml = convert_to_musicxml(file, key_to_convert)
    
    out_file_name = os.path.basename(file).split(".")[0] + ".musicxml"
    
    with open(os.path.join(save_dir, out_file_name), "w") as outfile:
        outfile.write(musicxml)

C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\1999_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\all_along_the_watchtower_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\all_apologies_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\all_i_have_to_do_is_dream_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\anarchy_in_the_uk_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\a_change_is_gonna_come_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\a_day_in_the_life_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\a_hard_days_night_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\a_whiter_shade_of_pale_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\back_in_black_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\be-bop-a-lula_dt.mel
C:\Users\Danie\

C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\i_want_you_back_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\jailhouse_rock_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\johnny_b_goode_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\jumpin_jack_flash_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\kashmir_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\layla_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\lets_get_it_on_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\lets_stay_together_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\let_it_be_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\light_my_fire_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\like_a_rolling_stone_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_

C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\whatd_i_say_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\whats_going_on_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\when_a_man_loves_a_woman_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\when_doves_cry_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\while_my_guitar_gently_weeps_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\whole_lotta_love_tdc.mel
Encountered unclear meter, defaulting to most recent time signature
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\whole_lotta_shakin_goin_on_tdc.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\who_do_you_love_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\will_you_love_me_tomorrow_dt.mel
C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_melody\with_or_without_you_dt.mel
C:\Users\Danie\P